# CellDancer benchmark on cell cycle data

Use cellDancer inferred cell specific transcription rate and compare with ground truth.

Note that cellDancer requires `anndata == 0.8.0`

## Library imports

In [5]:
import celldancer as cd
import celldancer.cdplt as cdplt
from celldancer.cdplt import colormap

import scanpy as sc
from rgv_tools import DATA_DIR

## Constants

In [7]:
DATASET = "cell_cycle_rpe1"

In [ ]:
SAVE_DATA = True
if SAVE_DATA:
    (DATA_DIR / DATASET / "processed").mkdir(parents=True, exist_ok=True)
    (DATA_DIR / DATASET / "results").mkdir(parents=True, exist_ok=True)

## Data loading

In [10]:
adata = sc.read_h5ad(DATA_DIR / DATASET / "processed" / "adata_processed.h5ad")

In [12]:
cell_type_u_s = cd.adata_to_df_with_embed(
    adata,
    cell_type_para="cell_cycle_phase",
    save_path=DATA_DIR / DATASET / "processed" / "cell_type_u_s_sample_df_processed.csv",
)

100%|██████████| 141/141 [00:02<00:00, 53.48it/s]


In [13]:
loss_df, cellDancer_df = cd.velocity(
    cell_type_u_s, permutation_ratio=0.1, norm_u_s=False, norm_cell_distribution=False, n_jobs=8
)

Using /ictstr01/home/icb/weixu.wang/regulatory_velo/regvelo_revision/cell_cycle_REF/cellDancer_velocity_2025-08-19 17-14-48 as the output path.
Arranging genes for parallel job.
141  genes were arranged to  18  portions.



Velocity Estimation: 100%|██████████| 18/18 [01:13<00:00,  4.97s/it]
                                                                    

In [16]:
alpha_matrix = cellDancer_df.pivot(index="cellIndex", columns="gene_name", values="alpha")

In [17]:
alpha_matrix.index = adata.obs_names

## Save data

In [18]:
if SAVE_DATA:
    DATA_DIR / DATASET / "processed" / "cell_type_u_s_sample_df_processed.csv"
    alpha_matrix.to_csv(DATA_DIR / DATASET / "processed" / "celldancer_alpha_estimate_processed.csv")